<a href="https://colab.research.google.com/github/Shakespeare1998/Sales-Tax-RGV/blob/main/Sales_Tax_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
from datetime import timedelta

uploaded = files.upload()

Saving Sales_Tax_Allocation__City-County_Comparison_Summary_20241203.csv to Sales_Tax_Allocation__City-County_Comparison_Summary_20241203.csv


In [2]:
# prompt: read csv file as pandas df

import pandas as pd
import io

df = pd.read_csv('/content/Sales_Tax_Allocation__City-County_Comparison_Summary_20241203.csv')
#df.head()

df['Date'] = df['Report Month'].apply(lambda x: f"{x:02}") + "-" + df['Report Year'].astype(str)
df['Date'] = pd.to_datetime(df['Date'], format='%m-%Y', errors='coerce')

df = df.drop(['Report Month', 'Report Year', 'Report Period Type', 'Payment to Date', 'Previous Payments to Date', 'Percent Change To Date'], axis=1)

In [3]:
city_list = ['El Paso', 'Del Rio', 'Eagle Pass', 'Laredo', 'Roma', 'Rio Grande City', 'La Joya', 'Penitas', 'Palmhurst', 'Palmview', 'Mission', 'Hidalgo', 'Alton', 'McAllen', 'Edinburg', 'Pharr', 'San Juan', 'Alamo', 'Donna', 'Elsa', 'Edcouch', 'Weslaco', 'Progreso', 'Mercedes', 'La Feria', 'Combes', 'Rio Hondo', 'Harlingen', 'San Benito', 'Los Fresnos', 'Port Isabel', 'Brownsville', 'South Padre Island', 'Raymondville']

df['Date'] = df['Date'] - pd.DateOffset(months=2)
df = df[df['Date'] > '2023-12-01']
df = df[df['City'].isin(city_list)]

df = df.rename(columns={'Current Rate': 'Sales Tax Rate', 'Net Payment This Period': '2024', 'Comparable Payment Prior Year': '2023', 'Percent Change From Prior Year': 'Percent Change'})
df['Difference'] = df['2024'] - df['2023']
df['Month'] = df['Date'].dt.strftime('%B')

df = df.drop(['Date', 'County'], axis=1)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[:-2] + cols[-1:] + cols[-2:-1]
df = df[cols]


city_dfs = {}
for city in city_list:
    city_dfs[city] = df[df['City'] == city].copy()
    city_dfs[city] = city_dfs[city].reset_index(drop=True)

In [4]:
city_dfs['El Paso']

,Month,City,Sales Tax Rate,2024,2023,Difference,Percent Change
0,January,El Paso,1.0,10100723.25,9883311.33,217411.92,2.19
1,February,El Paso,1.0,10090361.48,9939568.17,150793.31,1.51
2,March,El Paso,1.0,11734602.24,11926333.95,-191731.71,-1.60
3,April,El Paso,1.0,10649009.89,10451765.44,197244.45,1.88
4,May,El Paso,1.0,11035708.71,11122552.13,-86843.42,-0.78
5,June,El Paso,1.0,11569270.10,12140870.61,-571600.51,-4.70
6,July,El Paso,1.0,12026821.35,10738491.66,1288329.69,11.99
7,August,El Paso,1.0,10423553.62,10321726.03,101827.59,0.98
8,September,El Paso,1.0,10154525.89,11065222.32,-910696.43,-8.23


In [5]:
# prompt: loop through city_dfs dict to return city and df
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
n = 0

summary_data = []
for city, df_c in city_dfs.items():
    #print(f"City: {city}")
    total_2024 = df_c['2024'].sum()
    total_2023 = df_c['2023'].sum()
    total_diff = total_2024 - total_2023
    total_diff = round(total_diff, 2)
    percentage_diff = (total_diff / total_2023) * 100 if total_2023 != 0 else 0  # Handle potential ZeroDivisionError
    percentage_diff = round(percentage_diff, 2)

    summary_data.append([city, total_2024, total_2023, total_diff, percentage_diff])

    df_c['YTD - 2024'] = df_c['2024'].cumsum()
    #print(df_c)
    #print("-" * 20)
    interest_rate = df_c['Sales Tax Rate'].astype(float).mean()
    df_c = df_c.drop(['Sales Tax Rate', 'City'], axis=1)

    if n == 0:
        wb = Workbook()
        ws = wb.active
        ws.title = city
        ws.append(["City", city, "", "", "Interest Rate", interest_rate])
        ws.append([])
        n += 1

    else:
        ws = wb.create_sheet(title=city)
        ws.append(["City", city, "", "", "Interest Rate", interest_rate])
        ws.append([])

    for row in dataframe_to_rows(df_c, index=False, header=True):
        ws.append(row)

summary_df = pd.DataFrame(summary_data, columns=['City', '2024 Total', '2023 Total', 'Total Difference', 'Difference (%)'])
ws = wb.create_sheet(title="Summary")
ws.append(["Sales Tax Totals - 2024 vs 2023 (Jan to Sep)"])
ws.append([])

for row in dataframe_to_rows(summary_df, index=False, header=True):
    ws.append(row)

file_path = "Sales Tax (2024 vs 2023).xlsx"
wb.save(file_path)